# Person Motion Detection

In this notebook we combine our person detection/counter with the motion detection, so that we can detect which person is moving.

In [ ]:
# Import libraries
from jetbot import Camera
from jetbot import bgr8_to_jpeg
from jetbot import ObjectDetector

import cv2
import numpy as np
import time
from PIL import Image
import imutils

from IPython.display import display
import ipywidgets.widgets as widgets

In [ ]:
# Load mobilenet-v2 pretrained on COCO
model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

In [ ]:
# Init camera with same resolution as model input (300x300)
camera = Camera.instance(width=300, height=300)

In [98]:
# Define functions

def detect_people(object_detector, image, conf_thr):
    """
    Detect people on an image and return bounding boxes.

    Arguments:
        net: ObjectDetector model
        image (array): input image
        conf_thr (float): confidence threshold
        
    Returns:
            list: of bounding boxes (left, top, right, bottom)
    """

    person_class = 1

    # Image size
    rows = 300
    cols = 300

    # Make prediction on image
    detections = object_detector(image)

    # Iterate over each detection and save boundig box 
    # if confidence is above threshold and detected class is person
    person_boxes = []

    for detection in detections[0]:
            if detection['confidence'] > conf_thr and detection['label'] == person_class:
                    left = int(detection['bbox'][0] * cols)
                    top = int(detection['bbox'][1]  * rows)
                    right = int(detection['bbox'][2] * cols)
                    bottom = int(detection['bbox'][3]  * rows)
                    person_boxes.append([left, top, right, bottom])

    return sorted(person_boxes)

# Generate a random color palette
COLORS = np.random.uniform(0, 255, size=(15, 3))


def plot_boxes(image, people_boxes, motion_boxes=[]):
    """
    Plot bounding boxes on an image.

    Arguments:
        image (array): input image
        people_boxes (array): array of people bounding boxes with [left, top, right, bottom] positions
        motion_boxes (array): array of motion bounding boxes with [left, top, right, bottom] positions
    """
    # Plot people bounding boxes and corresponding number
    for i in range(len(people_boxes)):
        bbox = people_boxes[i]

        left = bbox[0]
        top = bbox[1]
        right = bbox[2]
        bottom = bbox[3]

        # Plot bounding box
        cv2.rectangle(image, (left, top), (right, bottom), COLORS[i], thickness=2)
        cv2.putText(image, f'{i+1}', (int(left)+5, int(top)+20), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 2)

    # Plot motion bounding boxes in green
    for bbox in motion_boxes:
        left = bbox[0]
        top = bbox[1]
        right = bbox[2]
        bottom = bbox[3]

        # Plot bounding box
        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), thickness=1)
        
    return image
 
    
def display_image(image, image_widget, text1="", text2=""):
    """
    Display an image on a Jupyter Widget and optionally puts top/bottom text.
    
    Arguments:
        image (array): image to display.
        img_widget (widgets.Image, optional): widget used to display the image
        text1 (str, optional): Optional 1st text to print on the top of image.
        text2 (str, optional): Optional 2nd text to print on the top of image.
    """
                
    # Add optional text
    if text1:
        cv2.putText(image, text1, (10, 20), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 2)
    
    if text2:
        cv2.putText(image, text2, (10, 40), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)
        
    # Display image
    image_jpeg = bgr8_to_jpeg(image)
    image_widget.value = image_jpeg


def motion_detection(firstFrame, newFrame, threshold=40):
    """
    Detect motion by comparing the newFrame with a firstFrame.
    
    Arguments:
        firstFrame (array): first frame considered as the baseline.
        newFrame (array): new frame.
        threshold (int): movement threshold.
        
    Returns:
        list: of regions where movement has been detected (left, top, right, bottom)
    """
    # Adapted from https://www.pyimagesearch.com/2015/05/25/basic-motion-detection-and-tracking-with-python-and-opencv/

    # Min movement area
    MIN_AREA = 50

    # Gaussian blur kernel
    KERNEL_SIZE = 3

    # List to store regions where movement is detected
    motion_bboxes = []
    
    # firstFrame preprocessing
    firstFrame = cv2.cvtColor(firstFrame, cv2.COLOR_BGR2GRAY)
    firstFrame = cv2.GaussianBlur(firstFrame, (KERNEL_SIZE, KERNEL_SIZE), 0)
    
    # grab the current frame and initialize the static/moving text
    frame = newFrame.copy()

    # convert frame to grayscale, and blur it
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (KERNEL_SIZE, KERNEL_SIZE), 0)

    # compute the absolute difference between the current frame and
    # first frame
    frameDelta = cv2.absdiff(firstFrame, gray)
    thresh = cv2.threshold(frameDelta, threshold, 255, cv2.THRESH_BINARY)[1]

    # dilate the thresholded image to fill in holes, then find contours
    # on thresholded image
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)   
    cnts = imutils.grab_contours(cnts)

    # loop over the contours
    for c in cnts:
        # if the contour is too small, ignore it
        if cv2.contourArea(c) < MIN_AREA:
            continue

        # compute the bounding box for the contour, draw it on the frame,
        # and update the text
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)
        
        motion_bboxes.append((x, y, x+w, y+h))    # (left, top, right, bottom)
    
    return motion_bboxes

def box_in_box(in_box, out_box, margin=0):
    """
    Evaluate if a bounding box is partially inside another one.
    
    Arguments:
        in_box (array): inner bounding box (left, top, right, bottom).
        out_box (array): outter bounding box (left, top, right, bottom).
        margin (int, optional): percentual margin to add to outter box. Defaults to 0.
        
    Returns:
        boolean
    """
    top_left = False
    bottom_right = False
    
    # Apply margin
    out_box[0] = int((1 - margin/100) * out_box[0])
    out_box[1] = int((1 - margin/100) * out_box[1])
    out_box[2] = int((1 + margin/100) * out_box[2])
    out_box[3] = int((1 + margin/100) * out_box[3])
    
    # Top-left corner
    if out_box[0] <= in_box[0] <= out_box[2] and out_box[1] <= in_box[1] <= out_box[3]:
        top_left = True
        
    # Bottom-right corner
    if out_box[0] <= in_box[2] <= out_box[2] and out_box[1] <= in_box[3] <= out_box[3]:
        bottom_right = True
            
    # Return True if top-left and bottom-right corners are inside
    if top_left and bottom_right:
        return True
    
def allow_movement(jetbot_camera, image_widget, detection_time, video_writer=None):
    """
    Perform person detection and display results.
    
    Arguments:
        jetbot_camera(Camera): Jetbot camera initialized with size (300, 300).
        image_widget (widgets.Image): Jupyter image widget.
        video_writer (cv2.VideoWriter, optional): OpenCV video writer used to save output.
        
    """
    initial_t = time.time()
    elapsed_time = 0

    while elapsed_time < detection_time:
        # Make people detections
        img = jetbot_camera.value.copy()
        people_detections = detect_people(model, img, 0.1)
        
        # Display image and detections
        image = plot_boxes(img, people_detections)
        display_image(image, image_widget, 'MOVE', f'{int(detection_time - elapsed_time)}s')
        
        # Save video
        if video_writer:
            video_writer.write(image)
            
        elapsed_time =  time.time() - initial_t

    
def detect_person_motion(jetbot_camera, image_widget, detection_time=5, threshold=40, video_writer=None):
    """
    Perform person and motion detection in a detection window of detection_time
    and display results.
    
    Arguments:
        jetbot_camera(Camera): Jetbot camera initialized with size (300, 300).
        image_widget (widgets.Image): Jupyter image widget.
        detection_time(int, optional): Total to perform motion detection.
        threshold (int): movement threshold.
        video_writer (cv2.VideoWriter, optional): OpenCV video writer used to save output.
    """    
    # Get first frame
    first_frame = jetbot_camera.value.copy()

    initial_t = time.time()
    elapsed_time = 0
    t = ""

    # Make people detections
    people_detections = detect_people(model, jetbot_camera.value.copy(), 0.1)
    
    # Run motion detection for MAX_TIME
    moving_persons_total = []
    while elapsed_time < detection_time:
        # Get new frame
        img = jetbot_camera.value.copy()
        
        # Detect motion
        m_bboxes = motion_detection(first_frame, img, threshold)        
        m_bboxes = set(m_bboxes)    # remove duplicates
        
        # Detect which person is moving
        moving_persons = []
        for i in range(len(people_detections)):
            out_box = people_detections[i]                
            person_num = i+1
            
            for in_box in m_bboxes:
                if box_in_box(list(in_box), out_box):
                    if person_num not in moving_persons:
                        moving_persons.append(person_num)
                        
        moving_persons_total.extend(moving_persons)
        
        # Persons moved in this frame
        if len(moving_persons):
            str_nums = ", ".join([str(i) for i in moving_persons])
            t = f'Moving: {str_nums}'
        
        # Display image and detections
        image = plot_boxes(img, people_detections, m_bboxes)
        display_image(image, image_widget, 'STOP', t)
        
        # Save video
        if video_writer:
            video_writer.write(image)
            
        elapsed_time =  time.time() - initial_t

    # Print persons moved in detection window
    moving_persons_total = set(moving_persons_total)
    if len(moving_persons_total):
        str_nums = ", ".join([str(i) for i in moving_persons_total])
        print(f'Person(s) moving: {str_nums}')

In [55]:
# Image widget
image_widget = widgets.Image(format='jpeg', width=400, height=400)
display(image_widget)

Image(value=b'', format='jpeg', height='400', width='400')

In [97]:
time.sleep(10)

stop_time = 3
move_time = 6

# Video Writer
cv2_video_writer = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (300,300))
# cv2_video_writer = None

# Main loop
print('Start')
while True:
    try:
        # MOVE - Allow movement
        allow_movement(camera, image_widget, move_time, cv2_video_writer)

        # STOP - Detect movement
        detect_person_motion(camera, image_widget, stop_time, video_writer=cv2_video_writer)
    
    except KeyboardInterrupt:
        break

print('Finish')        
cv2_video_writer.release()

Start
Person(s) moving: 1
Person(s) moving: 1
Finish
